# Schiphol API

## Metadata

- **Vak**: ID Datavisualisatie
- **Workshop**: Datavis API Workshop
- **Lecturers**: Laura Benvenuti & Danny de Vries
- **University**: Amsterdam University of Applied Sciences
- **Programme**: Communication and Multimedia Design
- **Faculty**: Digital Media and Creative Industries

## Omschrijving

Dit is een wat complexer notebook, een beetje een 'typisch' notebook die je zou verwachten als je data uit een api haalt en wilt plotten. De Schiphol API heeft authenticatie met een app key en meerdere endpoints om data uit verschillende datasets te halen. 

## Taken

Dit notebook doet:

* Een .csv met een export vanuit qualtrics inladen.
* Opschonen tot alleen relevante data, weghalen van metadata zoals tijdstip. locatie etc.
* Verwijderen van respondenten die de survey niet hebben afgemaakt.
* Een schone .csv naar een map op je computer schrijven.

### Check python versie

In [10]:
from packaging import version
import platform
import sys

min_version = '3.8'

def check_version(min_version):
    current_version = sys.version.split()[0]
    return version.parse(current_version) >= version.parse(min_version)

# Example usage:
if __name__ == "__main__":
    if check_version(min_version):
        print("Running a sufficiently new version of Python.")
        print("Current version: " + platform.python_version())
        print("Minimum required version: " + min_version)
    else:
        print("Python version is too old. Upgrade to a newer version.")

Running a sufficiently new version of Python.
Current version: 3.12.5
Minimum required version: 3.8


### Packages installeren

In [11]:
!pip install importlib
!pip install requests
!pip install pandas
!pip install tabulate
!pip install geopy
!pip install plotly
!pip install python-dotenv
!pip install nbformat

### Importeer packages

In [12]:
import subprocess
import importlib.util
import requests
import sys
import os
import nbformat
import pandas as pd
from tabulate import tabulate
from geopy.geocoders import Nominatim
import plotly.graph_objs as go
from dotenv import load_dotenv


### Data ophalen van verschillende API endpoints

In [13]:
# Load environment variables from .env file
load_dotenv()


def fetch_all_data(url_template, key):
    headers = {
        'accept': 'application/json',
        'resourceversion': 'v4',
        'app_id': os.getenv('SCHIPHOL_APP_ID'),
        'app_key': os.getenv('SCHIPHOL_APP_KEY')
    }

    all_data = []
    page = 1  # Start with page 1

    
    while page <= 1: #You can change this number to retrieve more pages but for this proof of concept we're only using the first page
        try:
            url = url_template.format(page)
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            page_data = data.get(key, [])
            if not page_data:
                break  # If no more data in the page, exit the loop
            all_data.extend(page_data)
            page += 1  # Move to the next page
        except requests.exceptions.RequestException as error:
            print(error)
            sys.exit()  # If an error occurs, print error message and exit the program

    return all_data

if __name__ == '__main__':
    date = '2024-02-14'  # Set date for which the data is retrieved, this can be any other date

    # Define all the four endpoints
    endpoints = {
        'flights': {'url': f'https://api.schiphol.nl/public-flights/flights?scheduleDate={date}', 'key': 'flights'},
        'aircraft_types': {'url': 'https://api.schiphol.nl/public-flights/aircrafttypes', 'key': 'aircraftTypes'},
        'airlines': {'url': 'https://api.schiphol.nl/public-flights/airlines', 'key': 'airlines'},
        'destinations': {'url': 'https://api.schiphol.nl/public-flights/destinations', 'key': 'destinations'}
    }

    # Fetch data from each endpoint 
    all_data = {}
    for endpoint, config in endpoints.items():
        all_data[endpoint] = fetch_all_data(config['url'], config['key'])

    # Create dataframes for each endpoint using pandas
    dfs = {}
    for endpoint, data in all_data.items():
        dfs[endpoint] = pd.DataFrame(data)

  
    # Print a sample from each DataFrame to check if it all worked 
    print("Flights DataFrame:")
    print(dfs['flights'].head())

    print("\nAircraft Types DataFrame:")
    print(dfs['aircraft_types'].head())

    print("\nAirlines DataFrame:")
    print(dfs['airlines'].head())

    print("\nDestinations DataFrame:")
    print(dfs['destinations'].head())


Flights DataFrame:
                   lastUpdatedAt              actualLandingTime  \
0  2024-02-14T02:01:20.213+01:00  2024-02-13T23:49:52.000+01:00   
1  2024-02-14T02:01:20.213+01:00  2024-02-13T23:49:52.000+01:00   
2  2024-02-14T02:00:00.196+01:00  2024-02-14T00:16:29.000+01:00   
3  2024-02-14T02:00:00.196+01:00  2024-02-14T00:16:29.000+01:00   
4  2024-02-14T02:00:00.196+01:00  2024-02-14T00:16:29.000+01:00   

                            aircraftType       baggageClaim  \
0  {'iataMain': '737', 'iataSub': '73H'}   {'belts': ['6']}   
1  {'iataMain': '737', 'iataSub': '73H'}   {'belts': ['6']}   
2  {'iataMain': '737', 'iataSub': '73H'}  {'belts': ['15']}   
3  {'iataMain': '737', 'iataSub': '73H'}  {'belts': ['15']}   
4  {'iataMain': '737', 'iataSub': '73H'}  {'belts': ['15']}   

                             codeshares           estimatedLandingTime  \
0            {'codeshares': ['KL2746']}  2024-02-13T23:50:32.000+01:00   
1            {'codeshares': ['KL2746']}  2024-02-13

## Samenvoegen van data

In [14]:
# Rename the 'publicName' column from the airline DataFrame to 'Airline'
dfs['airlines'].rename(columns={'publicName': 'airline'}, inplace=True)

# Rename the 'publicName' column from the destinations DataFrame to 'publicName destination'
dfs['destinations'].rename(columns={'publicName': 'publicName destination'}, inplace=True)

# Merge dataframes based on index
merged_df = pd.concat([dfs['flights'], dfs['aircraft_types'], dfs['airlines'], dfs['destinations']], axis=1)

# Print a sample of the merged dataframe
print("Sample of the Merged DataFrame:")
print(merged_df.head())


Sample of the Merged DataFrame:
                   lastUpdatedAt              actualLandingTime  \
0  2024-02-14T02:01:20.213+01:00  2024-02-13T23:49:52.000+01:00   
1  2024-02-14T02:01:20.213+01:00  2024-02-13T23:49:52.000+01:00   
2  2024-02-14T02:00:00.196+01:00  2024-02-14T00:16:29.000+01:00   
3  2024-02-14T02:00:00.196+01:00  2024-02-14T00:16:29.000+01:00   
4  2024-02-14T02:00:00.196+01:00  2024-02-14T00:16:29.000+01:00   

                            aircraftType       baggageClaim  \
0  {'iataMain': '737', 'iataSub': '73H'}   {'belts': ['6']}   
1  {'iataMain': '737', 'iataSub': '73H'}   {'belts': ['6']}   
2  {'iataMain': '737', 'iataSub': '73H'}  {'belts': ['15']}   
3  {'iataMain': '737', 'iataSub': '73H'}  {'belts': ['15']}   
4  {'iataMain': '737', 'iataSub': '73H'}  {'belts': ['15']}   

                             codeshares           estimatedLandingTime  \
0            {'codeshares': ['KL2746']}  2024-02-13T23:50:32.000+01:00   
1            {'codeshares': ['KL2746']

### Landen van vluchten ophalen

In [15]:

# Summary of the destination countries 
country_summary = merged_df['country'].value_counts()

# Convert to APA format
country_summary_apa = country_summary.to_frame().to_markdown(tablefmt="pipe")

# Check what the most used aircraft type is
most_used_aircraft_type = merged_df['aircraftType'].value_counts().idxmax()

print("\nSummary of destinations:")
print(country_summary_apa)
print("\nMost Used Aircraft Type:", most_used_aircraft_type)



Summary of destinations:
| country        |   count |
|:---------------|--------:|
| Danmark        |       2 |
| USA            |       2 |
| Brazil         |       2 |
| French Polynes |       1 |
| Indonesia      |       1 |
| Russia (CIS)   |       1 |
| Venezuela      |       1 |
| United Arab Em |       1 |
| South Africa   |       1 |
| Surinam        |       1 |
| Kiribati       |       1 |
| Australia      |       1 |
| Germany        |       1 |
| Algeria        |       1 |
| Sudan          |       1 |
| Egypt          |       1 |
| China          |       1 |

Most Used Aircraft Type: {'iataMain': '737', 'iataSub': '73H'}


### Vertragingen van vluchten.

In [16]:
# Convert 'estimatedLandingTime' and 'actualLandingTime' to datetime objects
merged_df['estimatedLandingTime'] = pd.to_datetime(merged_df['estimatedLandingTime'])
merged_df['actualLandingTime'] = pd.to_datetime(merged_df['actualLandingTime'])

# Calculate arrival_delay
merged_df['arrival_delay'] = merged_df['estimatedLandingTime'] - merged_df['actualLandingTime']

# Sort DataFrame based on delay in days
sorted_df = merged_df.sort_values(by='arrival_delay', ascending=False)

# Extract top 3 rows with highest delay
top_3_delayed_airlines = sorted_df[['actualLandingTime', 'estimatedLandingTime', 'country', 'airline', 'arrival_delay']].head(3)

# Convert DataFrame to APA 7 table format
apa_table = tabulate(top_3_delayed_airlines, headers='keys', tablefmt='pipe', showindex=False)

# Display the APA 7 formatted table
print("Top 3 Delayed Airlines:")
print(apa_table)

print(f"Maybe reconsider flying with {top_3_delayed_airlines['airline'].values.tolist()} if you want to be sure your flight is on time.")

Top 3 Delayed Airlines:
| actualLandingTime         | estimatedLandingTime      | country        | airline   | arrival_delay   |
|:--------------------------|:--------------------------|:---------------|:----------|:----------------|
| 2024-02-14 06:04:03+01:00 | 2024-02-14 06:04:47+01:00 | Indonesia      | Air Anka  | 0 days 00:00:44 |
| 2024-02-14 05:34:08+01:00 | 2024-02-14 05:34:51+01:00 | Surinam        | Flyone    | 0 days 00:00:43 |
| 2024-02-13 23:49:52+01:00 | 2024-02-13 23:50:32+01:00 | French Polynes | Blue Air  | 0 days 00:00:40 |
Maybe reconsider flying with ['Air Anka', 'Flyone', 'Blue Air'] if you want to be sure your flight is on time.


## Landnamen omzetten naar lat en long

In [17]:
geolocator = Nominatim(user_agent="my_geocoder")

def get_coordinates(country):
    location = geolocator.geocode(country)
    if location:
        return (location.latitude, location.longitude)
    else:
        return (None, None)  # Return None for latitude and longitude if country not found

# Add the function to 'country' column to get coördinates
merged_df['latitude'], merged_df['longitude'] = zip(*merged_df['country'].apply(get_coordinates))

print(merged_df[['latitude', 'longitude', 'country']]) # Check if it worked

     latitude   longitude         country
0         NaN         NaN  French Polynes
1  -24.776109  134.755000       Australia
2   26.254049   29.267547           Egypt
3   10.900000    6.500000           Sudan
4   28.000027    2.999983         Algeria
5   39.783730 -100.445882             USA
6  -10.333333  -53.200000          Brazil
7   51.163818   10.447831         Germany
8  -10.333333  -53.200000          Brazil
9    4.141303  -56.077119         Surinam
10   0.344861  173.664177        Kiribati
11  55.670249   10.333328         Danmark
12 -28.816624   24.991639    South Africa
13        NaN         NaN  United Arab Em
14   8.001871  -66.110932       Venezuela
15  39.783730 -100.445882             USA
16  59.615664   91.162453    Russia (CIS)
17  55.670249   10.333328         Danmark
18  -2.483383  117.890285       Indonesia
19  35.000074  104.999927           China


### Visualiseren van de vluchten op een kaart

In [18]:
fig = go.Figure()

# Loop through each flight entry to add line from Netherlands to destination
for country, lat, lon in zip(merged_df["country"], merged_df["latitude"], merged_df["longitude"]):
    fig.add_trace(go.Scattergeo(
        lat=[52.1326, lat],  # Latitude and longitude of the Netherlands because Schiphol is the origin
        lon=[5.2913, lon],   
        mode='lines',
        line=dict(color="pink", width=1),  # Make the lines thinner and pink
        name=country  # Set the trace name to the country name
    ))

# Edit layout
fig.update_layout(
    title=dict(
        text="Destination of flights from Schiphol on Valentines day 💘 ",
        font=dict(color="black")  #
    ),
    geo=dict(
        showland=True,
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)',
        projection_scale=1  # Increase the projection for larger layout
    ),
    showlegend=False,  # Delete legend
    height=800,  # Adjust the height of the box
    width=1000    # Adjust the width of the box
)

fig.show()